In [6]:
%config InlineBackend.figure_formats = ['svg']

import matplotlib.pyplot as plt
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Palatino"],
    "text.usetex": True,
    "font.size": 10
})



import sympy as sp 
from sympycalcs import render, convert, export, calcs
import sympy.physics.units as unit
from sympy.abc import *
import numpy as np
from IPython.display import Markdown
sp.init_printing(use_latex='mathjax', latex_mode='equation', mat_symbol_style = 'bold')



In [2]:
def antwortspektrum(a_gd,T, Baugrundklasse='A', eta=1):
    from sympy.physics.units import second
    from sympycalcs import render

    # Baugrundklassen
    Baugrundklassen = {
        'A': {
            'T_B':0.07*second,
            'T_C':0.25*second,
            'T_D':2.0*second,
            'S':1.0,
            'eta':eta,
        },
        'B': {
            'T_B':0.08*second,
            'T_C':0.35*second,
            'T_D':2.0*second,
            'S':1.2,
            'eta':eta,
        },
        'C': {
            'T_B':0.10*second,
            'T_C':0.4*second,
            'T_D':2.0*second,
            'S':1.45,
            'eta':eta,
        },
        'D': {
            'T_B':0.10*second,
            'T_C':0.5*second,
            'T_D':2.0*second,
            'S':1.7,
            'eta':eta,

        },
        'E': {
            'T_B':0.09*second,
            'T_C':0.25*second,
            'T_D':2.0*second,
            'S':1.7,
            'eta':eta,
        }
    }

    # Gewünschte Baugrundklasse auswählen
    params = Baugrundklassen[Baugrundklasse]
    T_B = params['T_B']
    T_C = params['T_C']
    T_D = params['T_D']
    S = params['S']
    
    render.dict_to_table(params)

    if T<=T_B:
        render.eq_display('S_e','a_gd*S_*(1+(2.5*eta_-1)*T/T_B)')
        return a_gd * S *(1+(2.5*eta-1)*T/T_B)
    
    if T_B<T<=T_C:
        render.eq_display('S_e','2.5*a_gd*S_*eta')
        return 2.5*a_gd*S*eta
    
    if T_C<T<=T_D:
        render.eq_display('S_e', '2.5*a_gd*S_*eta*T_C/T')
        return 2.5*a_gd*S*eta*T_C/T
    
    if T_D<=T:
        render.eq_display('S_e', '2.5*a_gd*S_*eta*T_C*T_D/T**2')
        return 2.5*a_gd*S*eta*T_C*T_D/T**2
    
    else:
        print('Falsche Eingabe')



In [3]:
def bemessungsspektrum(a_gd, T, q, gamma_f, Baugrundklasse='A'):
    from sympy.physics.units import second
    from sympycalcs import render
    
    g = 10.0*unit.m/unit.second**2
    
    # Baugrundklassen
    Baugrundklassen = {
        'A': {
            'T_B':0.07*second,
            'T_C':0.25*second,
            'T_D':2.0*second,
            'S':1.0,
            'g': g,
            'q': q,
            'gamma_f':gamma_f,
            'a_gd':a_gd
        },
        'B': {
            'T_B':0.08*second,
            'T_C':0.35*second,
            'T_D':2.0*second,
            'S':1.2,
            'g': g,
            'q': q,
            'gamma_f':gamma_f,
            'a_gd':a_gd

        },
        'C': {
            'T_B':0.10*second,
            'T_C':0.4*second,
            'T_D':2.0*second,
            'S':1.45,
            'g': g,
            'q': q,
            'gamma_f':gamma_f,
            'a_gd':a_gd
        },
        'D': {
            'T_B':0.10*second,
            'T_C':0.5*second,
            'T_D':2.0*second,
            'S':1.7,
            'g': g,
            'q': q,
            'gamma_f':gamma_f,
            'a_gd':a_gd

        },
        'E': {
            'T_B':0.09*second,
            'T_C':0.25*second,
            'T_D':2.0*second,
            'S':1.7,
            'g': g,
            'q': q,
            'gamma_f':gamma_f,
            'a_gd':a_gd
        }
    }

    # Gewünschte Baugrundklasse auswählen
    params = Baugrundklassen[Baugrundklasse]
    T_B = params['T_B']
    T_C = params['T_C']
    T_D = params['T_D']
    S = params['S']
    
    render.dict_to_table(params)

    if T<=T_B:
        render.eq_display('S_d','gamma_f*a_gd/g*S_*(0.67+(2.5/q-0.67)*T/T_B)')
        return gamma_f*a_gd/g*S*(0.67+(2.5/q-0.67)*T/T_B)
    
    if T_B<T<=T_C:
        render.eq_display('S_d','2.5*gamma_f*a_gd/g*S_/q')
        return 2.5*gamma_f*a_gd/g*S/q
    
    if T_C<T<=T_D:
        render.eq_display('S_d', '2.5*gamma_f*a_gd/g*S_*T_C/(T*q)')
        return 2.5*gamma_f*a_gd/g*S*T_C/(T*q)
    
    if T_D<=T:
        return max([2.5*gamma_f*a_gd/g*S*T_C*T_D/(T**2*q),0.1*gamma_f*a_gd/g])
    
    else:
        print('Falsche Eingabe')


In [4]:
def set_small_values_to_zero(matrix, threshold):
    """Setzt alle Werte in einer Sympy-Matrix, die kleiner als der Schwellenwert sind, auf Null.

    Args:
        matrix (Matrix): Die Sympy-Matrix
        threshold (float): Der Schwellenwert

    Returns:
        Matrix: Die modifizierte Matrix
    """
    from sympy import Matrix
    result = matrix.copy()
    for i in range(result.rows):
        for j in range(result.cols):
            if abs(result[i, j]) < threshold:
                result[i, j] = 0
    return result

In [5]:
def plot_einmassenschwinger(hoehen, massen, steifigkeiten, omegas, gammas):
    """Erstellt Plots für ein statisches System eines Einmassenschwingers für eine Liste von Höhen.

    Args:
        hoehen (list): Liste von Höhen
        massen (list): Liste von Massen
        steifigkeiten (list): Liste von Steifigkeiten
    """
    # Größenfaktor für die Massenpunkte berechnen
    min_masse = min(massen)
    groessenfaktor = 10 / min_masse

    # Anzahl der Subplots bestimmen
    n_subplots = len(hoehen)

    # Subplots erstellen
    fig, axs = plt.subplots(1, n_subplots)

    # Plots erstellen
    for i, (h, m, k, omega, gamma) in enumerate(zip(hoehen, massen, steifigkeiten, omegas, gammas)):
        # Stiel und Punktmasse zeichnen
        axs[i].plot([0, 0], [0, h], 'k-', linewidth=2)
        axs[i].plot([-0.2, -0.2], [0, h], 'k--', linewidth=1)

        axs[i].plot(0, h, 'o', markersize=m*groessenfaktor, markeredgecolor='black', markerfacecolor='lightgray')

        # Annotation hinzufügen
        axs[i].annotate(f'$k = {k}$ N/m\n$\Gamma = {gamma}$\n$m^\star = {m}$ kg', xy=(1, 1), xycoords='axes fraction', xytext=(-5, -5), textcoords='offset points', ha='right', va='top', fontsize=7)



        # Plot beschriften
        axs[i].set_title(f'$h = {h}$ m')
        axs[i].set_xlabel(f'$\omega = {omega}$ [1/s]')
        axs[i].set_xlim(-3, 3)
        axs[i].set_ylim(0, max(hoehen) + 1)
        axs[i].set_aspect('equal')
    plt.show()
